In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

labels_ = ['1', '2', '3', 'space', 'del', 'A', 'B', 'C']
NUM_CLASSES = len(labels_)
model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='relu', input_shape=(21, 3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# Create a checkpoint object and restore the weights
checkpoint_path = "model_checkpoints/cp_numbers_1.ckpt"
model.load_weights(checkpoint_path)

2023-05-09 18:34:51.226297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 18:34:58.116535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 21, 64)            17408     
                                                                 
 lstm_1 (LSTM)               (None, 21, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 8)                 264       
                                                                 
Total params: 172,136
Trainable params: 172,136
Non-trai

In [22]:
saved_model_path = "model/"
model.save(saved_model_path)


INFO:tensorflow:Assets written to: model/assets


In [23]:
loaded_model = tf.keras.models.load_model("model/")


In [24]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 21, 64)            17408     
                                                                 
 lstm_1 (LSTM)               (None, 21, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 8)                 264       
                                                                 
Total params: 172,136
Trainable params: 172,136
Non-trai

In [1]:
from fast_autocomplete import autocomplete_factory

content_files = {
    'words': {
        'filepath': 'word_dict.json',
        'compress': True  # means compress the graph data in memory
    }
}
alphabet = ['1', '2', '3', 'space', 'del', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
            'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
autocomplete = autocomplete_factory(content_files=content_files)


In [2]:
autocomplete.search(word='a',size=2, max_cost=100)

[['and'], ['as']]

In [4]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np

def most_frequent(List):
    return max(set(List), key=List.count)


In [ ]:
# initialze bounding box to empty
bbox = ''
count = 0

counter = 0

letters = []

limit_counter = 3

activate = 1

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
  while True:
      # js_reply = video_frame(label_html, bbox)
      # if not js_reply:
      #     break

      # # convert JS response to OpenCV Image
      # image = js_to_image(js_reply["img"])

      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = hands.process(image)

      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      landmarks = []
      letter = ''

      if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
              mp_drawing.draw_landmarks(
                  image,
                  hand_landmarks,
                  mp_hands.HAND_CONNECTIONS,
                  mp_drawing_styles.get_default_hand_landmarks_style(),
                  mp_drawing_styles.get_default_hand_connections_style())

              # Get the bounding box of the hand
              x_min, y_min, x_max, y_max = float('inf'), float(
                  'inf'), float('-inf'), float('-inf')
              for landmark in hand_landmarks.landmark:
                  x, y = landmark.x, landmark.y
                  x_min = min(x_min, x)
                  y_min = min(y_min, y)
                  x_max = max(x_max, x)
                  y_max = max(y_max, y)

              # Normalize the coordinates with respect to the bounding box of the hand
              for landmark in hand_landmarks.landmark:
                  x, y, z = landmark.x, landmark.y, landmark.z
                  x_norm = (x - x_min) / (x_max - x_min)
                  y_norm = (y - y_min) / (y_max - y_min)
                  landmarks.append([x_norm, y_norm, z])

          # Convert the landmarks to a feature vector
          x_t = np.array(landmarks)

          if x_t.flatten().shape[0] == 63 and counter < limit_counter and activate:

            counter += 1
            res_ = model.predict(np.array([x_t]), verbose=0)

            index_class = np.argmax(res_)

            letter_ = alphabet[index_class]

            letters.append(letter_)
          elif counter == limit_counter:
            letter = most_frequent(letters)

            if letter == 'space':
              print(' ', end='')
              print(autocomplete.search(word=letter, size=10, max_cost=100))
            else:
              print(letter_, end='')

            counter = 0

            activate = 0
      else:
          activate = 1
          counter = 0
          letters = []

      label_html = letter